In [ ]:
import os
from pathlib import Path
from types import SimpleNamespace

cfg = SimpleNamespace(**{})
cfg.num_folds = 5
cfg.gpu = "7"

cfg.fname = 'hms_102'
cfg.seed = 2024

cfg.input_path = Path('../input')
cfg.comp_data_path = cfg.input_path / 'birdclef-2024'
cfg.additional_data_path = cfg.input_path / 'birdclef2024-additional-mp3' / 'additional_audio'
cfg.save_path = Path('../checkpoints')

cfg.logger_file = True


cfg.sr = 32000

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from tqdm import tqdm
from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
import gc
import pickle as pkl

import librosa

from torch.utils.data import DataLoader, Dataset
import torchaudio
import torchaudio.transforms as T
#import torch_audiomentations as tA

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.cuda.amp import autocast, GradScaler
from torch.optim import lr_scheduler, Adam, AdamW

import timm

from glob import glob

In [ ]:
cfg.device = torch.device('cuda')

In [ ]:
file_paths = glob(str(cfg.additional_data_path) + "/*/*")
len(file_paths), file_paths[:5]

In [ ]:
train_additional = pd.DataFrame({
    'filename' : ['/'.join(filepath.split('/')[-2:]) for filepath in file_paths],
    'species' : [filepath.split('/')[-2] for filepath in file_paths],
    'record' : [filepath.split('/')[-1] for filepath in file_paths],
    'filepath': file_paths,
})
train_additional

In [ ]:
train = pd.read_csv(cfg.comp_data_path / 'train_metadata.csv')
train

train['species'] = [filename.split('/')[0] for filename in train.filename]
train['record'] = [filename.split('/')[1] for filename in train.filename]


In [ ]:
df = train.groupby('species').size()
df

In [ ]:
df1 = train_additional.groupby('species').size()
df1

In [ ]:
df = pd.concat([df, df1], axis=1)
df.columns = ['train', 'additional']
df

In [ ]:
df0 = df[df.train < 500].copy()

In [ ]:
_ = plt.scatter(df0.train, df0.additional, marker='+')

In [ ]:
train_additional = train_additional[train_additional.species.isin(set(df0.index))]
train_additional

In [ ]:
train_additional.to_csv(cfg.comp_data_path / 'train_additional.csv', index=None)